In [1]:
import mysql.connector
from mysql.connector import errorcode
import sqlalchemy
import pandas as pd
import matplotlib.pyplot as plt

### Database Setup
This portion populates a MySQL server with values from a pandas DataFrame.

In [2]:
# Run this the first time
!pip install pymysql

  Using cached PyMySQL-1.0.3-py3-none-any.whl (43 kB)



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Configure to your own user and password
mysql_user = 'root'
mysql_pw = 'password'
mysql_host = 'localhost'

engine = sqlalchemy.create_engine(f"mysql+pymysql://{mysql_user}:{mysql_pw}@{mysql_host}")
conn = engine.connect()
conn

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'root'@'localhost' (using password: YES)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
data = pd.read_csv("bechdel_analysis_data_cleaned.csv", float_precision=None)

In [ ]:
def execute_dml(dml_string):
    try:
        conn.execute(dml_string)
    except mysql.connector.Error as err:
        print("Error:", err)
        
def execute_query(query_string):
    try:
        res = conn.execute(query_string)
        return res.fetchall()
    except mysql.connector.Error as err:
        print("Error:", err)

def create_db():
    drop_existing_db = "DROP DATABASE IF EXISTS bechdel;"
    create_bechdel = "CREATE DATABASE bechdel;"
    use_bechdel = "USE bechdel;"
    try:
        execute_dml(drop_existing_db)
        execute_dml(create_bechdel)
        execute_dml(use_bechdel)
    except mysql.connector.Error as err:
        print("Error:", err)
        
def populate_db():
    general_data = data.drop(['genres', 'prod_comps'], axis=1)
    general_data.to_sql('general', conn, if_exists='replace', dtype={
        'year': sqlalchemy.types.INTEGER(),
        'imdb': sqlalchemy.types.VARCHAR(length=12),
        'title': sqlalchemy.types.VARCHAR(length=255),
        'clean_test': sqlalchemy.types.VARCHAR(length=20),
        'binary_passed': sqlalchemy.types.INTEGER(),
        'budget_2013$': sqlalchemy.types.BIGINT(),
        'domgross_2013$': sqlalchemy.types.BIGINT(),
        'intgross_2013$': sqlalchemy.types.BIGINT(),
        'overview': sqlalchemy.types.TEXT(),
        'perc_women_producers': sqlalchemy.types.FLOAT(),
        'perc_women_directors': sqlalchemy.types.FLOAT()
    })
    
    # Convert into workable list type before using .explode()
    genre_data = data[['imdb', 'genres']]
    for i in range(genre_data.count()[0]):
        genre_data.iloc[i].genres = genre_data.iloc[i].genres.strip('[]').replace('\'', "").split(', ')
    genre_data = genre_data.explode('genres')
    genre_data.to_sql('genre', conn, if_exists='replace', dtype={
        'imdb': sqlalchemy.types.VARCHAR(length=12),
        'genre': sqlalchemy.types.VARCHAR(length=20)
    })
    
    prod_data = data[['imdb', 'prod_comps']]
    for i in range(prod_data.count()[0]):
        prod_data.iloc[i].prod_comps = prod_data.iloc[i].prod_comps.strip('[]').replace('\'', "").split(', ')
    prod_data = prod_data.explode('prod_comps')
    prod_data.to_sql('production_company', conn, if_exists='replace', dtype={
        'imdb': sqlalchemy.types.VARCHAR(length=12),
        'genre': sqlalchemy.types.VARCHAR(length=50)
    })
    execute_dml('ALTER TABLE general ADD PRIMARY KEY (imdb);')
    execute_dml('ALTER TABLE genre ADD FOREIGN KEY (imdb) REFERENCES general(imdb);')
    execute_dml('ALTER TABLE production_company ADD FOREIGN KEY (imdb) REFERENCES general(imdb);')

In [ ]:
create_db()
populate_db()

print(execute_query("DESC general"), '\n');
print(execute_query("DESC genre"), '\n');
print(execute_query("DESC production_company"));

### Querying the Database

In [ ]:
print('Avg budget, domestic gross sales, and internationaly gross sales for movies that passed')

res = execute_query("""
    SELECT CONCAT('$', FORMAT(avg(budget_2013$), 2)),
           CONCAT('$', FORMAT(avg(domgross_2013$), 2)),
           CONCAT('$', FORMAT(avg(intgross_2013$), 2))
        FROM general
    WHERE binary_PASS = 1;
"""
)
print(res)

print('\nAvg budget, domestic gross sales, and internationaly gross sales for movies that failed')
res = execute_query("""
    SELECT CONCAT('$', FORMAT(avg(budget_2013$), 2)),
           CONCAT('$', FORMAT(avg(domgross_2013$), 2)),
           CONCAT('$', FORMAT(avg(intgross_2013$), 2))
        FROM general
    WHERE binary_PASS = 0;
"""
)
print(res[0])

In [ ]:
res = execute_query("""
    SELECT COUNT(*) AS movie_count,
           prod_comps,
           SUM(CASE WHEN g.binary_PASS = 1 THEN 1 ELSE 0 END) / COUNT(*) AS perc_passing
    FROM production_company pc
    JOIN general g ON pc.imdb = g.imdb
    GROUP BY prod_comps
    ORDER BY movie_count DESC
    LIMIT 100;
""")
prod_passing = pd.DataFrame(res)
print("Passing percentages for most popular production companies in dataset\n")
prod_passing

In [ ]:
plt.plot(prod_passing.movie_count, prod_passing.perc_passing)
plt.title("Passing Percentage by Production Company Movie Count")
plt.xlabel("Movies Produced by Company")
plt.ylabel("Percent of Movies Passing Bechdel Test")

In [ ]:
res = execute_query("""
    SELECT COUNT(*) AS movie_count,
           genres,
           SUM(CASE WHEN g.binary_PASS = 1 THEN 1 ELSE 0 END) / COUNT(*) AS perc_passing
    FROM genre gn
    JOIN general g ON gn.imdb = g.imdb
    GROUP BY genres
    ORDER BY movie_count DESC;
""")
genre_passing = pd.DataFrame(res)
print("Passing percentages by genre\n")
genre_passing.sort_values(by=['perc_passing'], ascending=False)